In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import shutil
import time
import os
from pyshadow.main import Shadow
from py7zr import unpack_7zarchive
from RequestInferSchemaToJsonAPI.main import TriggerInferShemaToJsonAPI
base_path = 'C:\\Users\\10235555\\Documents\\Dataportal Data\\ITU SDG indicators'

In [7]:
chrome_options = webdriver.ChromeOptions()
base_path = "C:\\Users\\10235555\\Documents\\Dataportal Data\\ITU SDG indicators" # local, gets current working directory
prefs = {'download.default_directory' : base_path}#, "profile.content_settings.exceptions.automatic_downloads.*.setting" : 1}
# prefs = {"download.default_directory": base_path , # pass the variable
#                    "download.prompt_for_download": False,
#                    "directory_upgrade": True,
#                    "safebrowsing.enabled": True }
chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_argument('--user-agent="Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; Lumia 640 XL LTE) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10166"')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

In [8]:
driver.get('https://www.itu.int/en/ITU-D/Statistics/Pages/SDGs-ITU-ICT-indicators.aspx')


In [9]:
datasets = driver.find_element(By.CLASS_NAME,'contentNew._invisibleIfEmpty')
list_datasets = datasets.find_element(By.TAG_NAME,'ul').find_elements(By.TAG_NAME,'li')
for elem in list_datasets:
    dataset = elem.text.replace(':',',')
    download = elem.find_element(By.TAG_NAME,'a')
    download.click()
    time.sleep(1)
    def latest_download_file():
        path = base_path
        os.chdir(path)
        files = sorted(os.listdir(os.getcwd()), key=os.path.getmtime)
        newest = files[-1]

        return newest

    fileends = "crdownload"
    while "crdownload" == fileends:
        time.sleep(5) 
        newest_file = latest_download_file()
        if "crdownload" in newest_file:
            fileends = "crdownload"
            # time.sleep(5)
        else:
            fileends = "None"
    latest_download_file() 
    file = max([base_path + "\\" + f for f in os.listdir(base_path)],key=os.path.getctime)
    filename = os.path.split(file)[1]
    print(dataset)
    print(filename)

Indicator 4.4.1, Proportion of youth and adults with ICT skills, by type of skills
2022_4.4.1_SE_ADT_ACTS_mn.xlsx
Indicator 5.b.1, Proportion of individuals who own a mobile telephone, by sex
2022_5.b.1.xlsx
Indicator 9.c.1, Proportion of population covered by a mobile network, by technology
2022_9.c.1_all_combined.xlsx
Indicator 17.6.1, Fixed Internet broadband subscriptions per 100 inhabitants, by speed
2022_17.6.1-BBP.xlsx
Indicator 17.8.1, Proportion of individuals using the Internet
2022_17.8.1_IT_USE_ii99.xlsx


In [15]:
notes = driver.find_elements(By.CLASS_NAME,'ms-rteTableEvenCol-3')
for note in notes:
    descr = note.text.split('The data can be found')[0]
    source = note.text.split('The data can be found')[1]
    source = 'The data can be found' + source
    # print(descr)
    print(source)

The data can be found in the World Telecommunication/ICT Indicators Database  and in the UN SDG Indicators Database. The metadata are also available on the UN website. For more SDG analysis, visit the UN Sustainable Developments Report 2021 or the Extended Report for SDG Goal 4.
The data can be found in the World Telecommunication/ICT Indicators Database and in the UN SDG Indicators Database. The metadata are also available on the UN website. For more SDG analysis, visit the UN Sustainable Developments Report 2021 or the Extended Report for SDG Goal 5.
The data can be found in the World Telecommunication/ICT Indicators Database  and in the UN SDG Indicators Database. The metadata are also available on the UN website. For more SDG analysis, visit the UN Sustainable Developments Report 2021 or the Extended Report for SDG Goal 9.
The data can be found in the World Telecommunication/ICT Indicators Database  and in the UN SDG Indicators Database. The metadata are also available on the UN we

In [28]:
import urllib.request
notes = driver.find_elements(By.CLASS_NAME,'ms-rteTableEvenCol-3')
for note in notes:
    links = note.find_elements(By.TAG_NAME,'a')
    for link in links:
        link_note = link.get_attribute('href')
        if link_note.endswith('.pdf'):
            if 'Metadata' in link_note:
                pdf = link_note
                pdf_name = pdf.split('files/')[1]
                urllib.request.urlretrieve(pdf,f"{base_path}\\{pdf_name}")
# links = driver.find_element(By.XPATH,"//*[contains(text(), 'metadata ')]")

In [29]:
conten=[]
import pdfplumber
with pdfplumber.open(f"{base_path}\\{pdf_name}") as pdf:

    text = pdf.pages
    for te in text:
        content = te.extract_text()
        conten.append(content)
content_all = ' '.join([str(elem) for elem in conten])

In [30]:
content_all

"Last updated: 2021-08-20 \nSDG indicator metadata\n \n(Harmonized metadata template - format version 1.1) \n \n0. Indicator information (SDG_INDICATOR_INFO) \n0.a. Goal (SDG_GOAL) \nGoal 17: Strengthen the means of implementation and revitalize the Global Partnership for Sustainable \nDevelopment \n0.b. Target (SDG_TARGET) \nTarget 17.8: Fully operationalize the technology bank and science, technology and innovation capacity-\nbuilding mechanism for least developed countries by 2017 and enhance the use of enabling technology, \nin particular information and communications technology \n0.c. Indicator (SDG_INDICATOR) \nIndicator 17.8.1: Proportion of individuals using the Internet \n0.d. Series (SDG_SERIES_DESCR) \nNot Applicable \n0.e. Metadata update (META_LAST_UPDATE) \n2021-08-20 \n0.f. Related indicators (SDG_RELATED_INDICATORS) \n1.4, 2.c, 4.3, 4.4, 5.b, 9.c, 10.3, 12.8, 16.10, 16.6, 16.7, 16.10, 17.6 \n0.g. International organisations(s) responsible for global monitoring \n(SDG_C

In [55]:
goal = content_all.split('0.a. Goal (SDG_GOAL) \n')[1].split('0.b. Target (SDG_TARGET) \n')[0].replace(' \n',' ')
target = content_all.split('0.b. Target (SDG_TARGET) \n')[1].split('0.c. Indicator (SDG_INDICATOR) \n')[0].replace(' \n',' ')
indicator = content_all.split('0.c. Indicator (SDG_INDICATOR) \n')[1].split('0.d. Series (SDG_SERIES_DESCR) \n')[0].replace(' \n',' ')
last_update = content_all.split('0.e. Metadata update (META_LAST_UPDATE) \n')[1].split('0.f. Related indicators (SDG_RELATED_INDICATORS) \n')[0].replace(' \n',' ')
definition = content_all.split('Definition: \n')[1].split('Concepts: \n')[0].replace(' \n',' ')
concept = content_all.split('Concepts: \n')[1].split('2.b. Unit of measure (UNIT_MEASURE) \n')[0].replace(' \n',' ').split('   Last updated: 2021-08-20 ')[0]
unit_measure = content_all.split('2.b.')[1].split('2.c.')[0].replace(' \n',' : ')
classification = content_all.split('2.c. Classifications (CLASS_SYSTEM) \n')[1].split('3. Data source type and data collection method \n')[0].replace(' \n',' ')
source = content_all.split('(SRC_TYPE_COLL_METHOD) \n')[1].split('4. Other methodological considerations (OTHER_METHOD) \n')[0].replace(' \n',' ')
coverage = content_all.split('5. Data availability and disaggregation (COVERAGE) \n')[1].split('6. Comparability / deviation from international standards \n')[0].replace(' \n',' ')

' Unit of measure (UNIT_MEASURE) : Percentage of individuals :  : '

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
import urllib.request
from bs4 import BeautifulSoup
import shutil
import time
import os
import pdfplumber
from pyshadow.main import Shadow
from py7zr import unpack_7zarchive
from RequestInferSchemaToJsonAPI.main import TriggerInferShemaToJsonAPI
base_path = 'C:\\Users\\10235555\\Documents\\Dataportal Data\\ITU SDG indicators'

chrome_options = webdriver.ChromeOptions()
base_path = "C:\\Users\\10235555\\Documents\\Dataportal Data\\ITU SDG indicators" # local, gets current working directory
prefs = {'download.default_directory' : base_path}#, "profile.content_settings.exceptions.automatic_downloads.*.setting" : 1}
# prefs = {"download.default_directory": base_path , # pass the variable
#                    "download.prompt_for_download": False,
#                    "directory_upgrade": True,
#                    "safebrowsing.enabled": True }
chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_argument('--user-agent="Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; Lumia 640 XL LTE) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10166"')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

driver.get('https://www.itu.int/en/ITU-D/Statistics/Pages/SDGs-ITU-ICT-indicators.aspx')
time.sleep(3)

datasets = driver.find_element(By.CLASS_NAME,'contentNew._invisibleIfEmpty')
list_datasets = datasets.find_element(By.TAG_NAME,'ul').find_elements(By.TAG_NAME,'li')
for elem in list_datasets:
    dataset = elem.text.replace(':',',')
    indicator_report = elem.text.split(':')[0]
    download = elem.find_element(By.TAG_NAME,'a')
    # download.click()
    time.sleep(1)
    def latest_download_file():
        path = base_path
        os.chdir(path)
        files = sorted(os.listdir(os.getcwd()), key=os.path.getmtime)
        newest = files[-1]

        return newest

    fileends = "crdownload"
    while "crdownload" == fileends:
        time.sleep(5) 
        newest_file = latest_download_file()
        if "crdownload" in newest_file:
            fileends = "crdownload"
            # time.sleep(5)
        else:
            fileends = "None"
    latest_download_file() 
    file = max([base_path + "\\" + f for f in os.listdir(base_path)],key=os.path.getctime)
    filename = os.path.split(file)[1]

    notes = driver.find_elements(By.CLASS_NAME,'ms-rteTableEvenCol-3')
    for note in notes:
        if indicator_report in note.text:
            links = note.find_elements(By.TAG_NAME,'a')
            for link in links:
                link_note = link.get_attribute('href')
                if link_note.endswith('.pdf'):
                    if 'Metadata' in link_note:
                        pdf = link_note
                        pdf_name = pdf.split('files/')[1]
                        urllib.request.urlretrieve(pdf,f"{base_path}\\{pdf_name}")
                        def latest_download_file():
                            path = base_path
                            os.chdir(path)
                            files = sorted(os.listdir(os.getcwd()), key=os.path.getmtime)
                            newest = files[-1]

                            return newest

                        fileends = "crdownload"
                        while "crdownload" == fileends:
                            time.sleep(5) 
                            newest_file = latest_download_file()
                            if "crdownload" in newest_file:
                                fileends = "crdownload"
                                # time.sleep(5)
                            else:
                                fileends = "None"
                        latest_download_file() 
                        conten=[]
                        with pdfplumber.open(f"{base_path}\\{pdf_name}") as pdf:
                            text = pdf.pages
                            for te in text:
                                content = te.extract_text()
                                conten.append(content)
                        content_all = ' '.join([str(elem) for elem in conten])
                        goal = content_all.split('0.a. Goal (SDG_GOAL) \n')[1].split('0.b. Target (SDG_TARGET) \n')[0].replace(' \n',' ')
                        target = content_all.split('0.b. Target (SDG_TARGET) \n')[1].split('0.c. Indicator (SDG_INDICATOR) \n')[0].replace(' \n',' ')
                        indicator = content_all.split('0.c. Indicator (SDG_INDICATOR) \n')[1].split('0.d. Series (SDG_SERIES_DESCR) \n')[0].replace(' \n',' ')
                        last_update = content_all.split('0.e. Metadata update (META_LAST_UPDATE) \n')[1].split('0.f. Related indicators (SDG_RELATED_INDICATORS) \n')[0].replace(' \n',' ')
                        try:
                            definition = content_all.split('Definition: \n')[1].split('Concepts: \n')[0].replace(' \n',' ')
                        except:
                            definition = content_all.split('Definitions: \n')[1].split('Concepts: \n')[0].replace(' \n',' ')
                        concept = content_all.split('Concepts: \n')[1].split('2.b. Unit of measure (UNIT_MEASURE) \n')[0].replace(' \n',' ').split('   Last updated: 2021-08-20 ')[0]
                        unit_measure = content_all.split('2.b.')[1].split('2.c.')[0].replace(' \n',' : ')
                        classification = content_all.split('2.c. Classifications (CLASS_SYSTEM) \n')[1].split('3. Data source type and data collection method \n')[0].replace(' \n',' ')
                        source = content_all.split('(SRC_TYPE_COLL_METHOD) \n')[1].split('4. Other methodological considerations (OTHER_METHOD) \n')[0].replace(' \n',' ')
                        coverage = content_all.split('5. Data availability and disaggregation (COVERAGE) \n')[1].split('6. Comparability / deviation from international standards \n')[0].replace(' \n',' ')
                        print(f'{dataset} : {goal} , {target} , {indicator} , {last_update} , {definition} , {concept} , {unit_measure} , {classification} , {source} , {coverage}')

Indicator 4.4.1, Proportion of youth and adults with ICT skills, by type of skills
Indicator 4.4.1
https://unstats.un.org/sdgs/metadata/files/Metadata-04-04-01.pdf
https://unstats.un.org/sdgs/metadata/files/Metadata-04-04-01.pdf
Indicator 5.b.1, Proportion of individuals who own a mobile telephone, by sex
Indicator 5.b.1
https://unstats.un.org/sdgs/metadata/files/Metadata-05-0b-01.pdf
Indicator 9.c.1, Proportion of population covered by a mobile network, by technology
Indicator 9.c.1
https://unstats.un.org/sdgs/metadata/files/Metadata-09-0c-01.pdf
Indicator 17.6.1, Fixed Internet broadband subscriptions per 100 inhabitants, by speed
Indicator 17.6.1
https://unstats.un.org/sdgs/metadata/files/Metadata-17-06-01.pdf
Indicator 17.8.1, Proportion of individuals using the Internet
Indicator 17.8.1
https://unstats.un.org/sdgs/metadata/files/Metadata-17-08-01.pdf
